# Introduction

This notebook allows you to run a simplified version of the ANTARESS workflow, for the following purpose:
- characterizing systematic trends in stellar lines properties during a night, using a CCF dataset

This notebook can take as input a time-series of DI echelle spectra (first cleaned with the [spectral reduction notebook](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_sp_reduc.ipynb)) or CCFs (possibly generated with the [mock notebook](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_mock.ipynb)). A typical use of the intrinsic CCFs output by this notebook is to analyze the Rossiter-McLaughlin signal from the transiting planet, which can be done in a simplified way using the [RM notebook](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_RMR.ipynb).  

The default version of this notebook is set up to process the default mock dataset provided with ANTARESS. To try it out, copy the directory `Arda/` from `Notebooks/Mock_system/` into the directory of your choice (named, for example, `Working_dir/`) and set `working_path` to this directory location. Otherwise adjust the settings to retrieve your own cleaned dataset, setting `working_path`, `star_name`, and `pl_name` to the same values that you used to initialize your [observed](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_setup.ipynb) or [mock](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_setup_Arda.ipynb) system.

To exploit the workflow to its full capabilities, run its executable with the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).











-    CCF time series, either CCFs computed in the Spectral reduction notebook or directly from the instrumental DRS pipeline.

- Investigate CCF properties and trends for the spectral line parameters
- Find the best-fitting model to correct for spectral line variations
- Apply trend correction for the specific spectral line properties

This notebook is used to process, detrend and scale CCF time series. To try the workflow, place your CCFs to process in your chosen directory (for example, `Working_dir/`) and set the `working_path` to the directory containing your data.

Some options and functionalities are not included in this notebook to keep it light and easy to use. For targets with multiple observations, it is especially recommended to use the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In [4]:
import ANTARESS_nbook_bground
input_nbook = ANTARESS_nbook_bground.init('Detrending')
input_nbook['working_path'] ='/Users/erikfriden/Desktop/antaress-pip/working_dir/'

TypeError: init() takes 0 positional arguments but 1 was given

# Data upload and formatting

# CCF Analysis

The first step in detrending will be to analyse the disk-integrated CCF profiles. Later, potential correlations for the line parameters full width at half maxima (FWHM), contrast, and RV will be analysed to see if they correlate with either time (phase) or SNR. For more options, use the configuration file.

**Fit disk-integrated CCF profiles**

The first step in fitting the CCF profiles will be to determine the velocity range for the fit and continuum. 
- `cont_range` (list, float): define regions used for the continuum, which can be multiple ranges
- `fit_range` (list, float): define range used for fit, can be multiple ranges and will then have the same format as `cont_range`


In [ ]:
input_nbook['DI_trend'].update({
    'cont_range' : [[-100., -25.], [25., 100.]],
    'fit_range'  : [[-100, 100]]
})

ANTARESS_nbook_bground.range_def(input_nbook)

**FIT Model parameters**

To fit the CCFs, we use a Gaussian model using $\chi^2$-minimization. The guess parameters will be used to initialise the starting values of the fitting routine. Choose guess values for RV and FWHM in km/s and the contrast. To plot the Gaussian fit set `plot_CCF` to true, the properties for RV, FWHM, and contrast will be plotted if `plot=True`.


In [ ]:
input_nbook['DI_trend'].update({
    'rv'       : 0.,
    'FWHM'     : 6.,
    'ctrst'    : 0.7,
    'plot_CCF' : True
})

ANTARESS_nbook_bground.fit_DI(input_nbook, plot=False)

<a id='fit_prop'></a>
**Trend characterisation**

Here, we investigate the data and the model parameters to identify any trends seen in the model parameters as either a function of time (phase) or SNR. The goal will be to find the model and its coefficients that best fit the data. No correction will be applied if a linear fit shows the best agreement with the data. The most common models to characterise the data are first or second-degree polynomials; if you need a more complex model to fit the data, e.g., a sinus model, you can use the full ANTARESS workflow using [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html). In this notebook, we only work with polynomials.

- `pol_deg` (integer): degree of polynomial to fit
- `x_var` (string): phase or SNR, choose what dependence to plot against for the properties (RV, FWHM, and ctrst).

Run the cell below to perform the fit for (RV, FWHM, and ctrst) as a function of `'phase'` or `'snr'`. Try different polynomial degrees. The Bayesian information criterion (BIC) will be printed for each fit to compare the different models.


In [ ]:
input_nbook['DI_trend'].update({
    'pol_deg' : 0.,
    'x_var'   : 'phase',
})

ANTARESS_nbook_bground.fit_prop(input_nbook, plot=False)

**Systemic velocity**

**If it is the first time you run the analysis, leave it as** `gamma = 0`. When you run ANTARESS the first time, you will measure the systemic velocity, which will be printed in the output, later set gamma to the measured value.

In [ ]:
input_nbook['par']['gamma'] = 0.
ANTARESS_nbook_bground.set_sysvel(input_nbook)

**Detrending of CCFs**

**If this is the first time processing the data set, skip this step and run ANTARESS in the cell below.** Identify any trends usinc the cell [Trend characterisation](#fit_prop), and find the best model representing the data. Once the best models for the parameters RV, FWHM, and contrast have been found using the Bayesian Information Criterion, insert the coefficient for each model below. If multiple properties are being corrected, add them to the list as shown in the example below.

- `use` (bool): set to True to perform a trend correction
- `prop` (list, str): list each property to correct with the corresponding dependence. Properties to correct here are either RV, FWHM, and contrast (ctrst), and as a function of either phase or snr
- `coeff` (list, list): indicate coefficients used for each model, add coefficients with decreasing order and add a new list if more than one property is corrected. The RV coefficient should be in km/s.

In [ ]:
input_nbook['par'].update({
    'use'  : False,
    'prop' : ['ctrst_snrQ', 'RV_phase'],
    'mode' : ['pol', 'pol'],
    'coeff': [[-1.109190e-05], [-6.840494e-02]]
})

ANTARESS_nbook_bground.detrend(input_nbook)

In [ ]:
from antaress.ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher
ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

# Display Plots

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

Run the cells below to show saved plots. Plots can be found in `working_path/targetName_plots/`. For the fits to the CCF go to `working_path/target/targetPL_Plots/DI_data/instrument_night_Indiv/Data/`

**Disk-integrated line properties**

The three plots below show the time-series of RVs, contrast, and FWHM derived from the fits to the disk-integrated CCFs. Transit contacts are shown as dashed black lines, and in-transit exposures are flagged with empty symbols.

Outside of the transit, RVs should trace the Keplerian motion of your star with respect to the solar system barycenter (the Keplerian model defined by your inputs for the star and planets is plotted as a solid blue curve). If your star is quiet and measurements are dominated by white noise, RV measurements should be evenly spread around the Keplerian model, while contrast and FWHM measurements should be evenly spread around their mean out-of-transit value (dashed blue lines). You can assess this by looking at the ratio between the dispersion and mean error over out-of-transit measurements, reported on each plot, which should be close to unity. If that is not the case, you can use the workflow to detrend the data.  

Inside of the transit the planet occults local regions of the stellar surface, distorting the disk-integrated stellar lines and inducing anomalous deviations in their derived properties. Traditionally, measurements of the sky-projected spin-orbit angle in planetary systems have been performed by analyzing this anomalous signal in RV - but the anomaly may not be detectable with faint, slow-rotating stars and small planets. With ANTARESS, you will be able to analyze directly the planet-occulted stellar profiles to get a higher precision on the transit chord and orbital architecture of the system.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_res_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/ctrst_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/FWHM_phase.png')